# Particionamento e Tabelas no Azure SQL

O **particionamento** de tabelas é uma técnica usada para dividir grandes volumes de dados em partes menores e mais gerenciáveis, chamadas de **partições**. No contexto do **Azure SQL Database** ou **Azure SQL Managed Instance**, particionar tabelas pode melhorar a performance das consultas, facilitar a manutenção e otimizar o uso de recursos ao trabalhar com grandes conjuntos de dados.  

<span style="background-color:transparent;font-size:14pt;font-family:&quot;Times New Roman&quot;, serif;color:rgb(0, 0, 0);vertical-align:baseline;"><br></span>

### Scripts executados nessa configuração
Microsoft SQL Server 2022 (RTM-GDR) (KB5042211) - 16.0.1125.1 (X64)   Jul 31 2024 23:58:42   Copyright (C) 2022 Microsoft Corporation  Developer Edition (64-bit) on Windows 10 Pro 10.0 <X64> (Build 22631: ) (Hypervisor) 


#### O que é o particionamento de tabelas?

# 

O particionamento envolve dividir uma tabela grande em segmentos menores (partições), baseando-se em uma coluna específica (como datas ou IDs). Cada partição pode ser armazenada de forma separada, mas elas ainda se comportam como uma única tabela do ponto de vista das consultas. Isso permite que o SQL Server processe as consultas de forma mais eficiente, acessando apenas as partições relevantes.

#### Vantagens do particionamento

# 

1. **Melhoria de desempenho**: Consultas que acessam uma pequena parte dos dados podem ser otimizadas, uma vez que o SQL Server só precisa ler as partições necessárias, reduzindo o I/O e o tempo de execução.
2. **Gerenciamento simplificado**: Tarefas de manutenção, como reconstrução de índices ou arquivamento de dados, podem ser realizadas em partições individuais, sem afetar a tabela inteira.
3. **Escalabilidade**: Ao particionar grandes volumes de dados, o armazenamento e a manutenção se tornam mais fáceis, permitindo que os bancos de dados cresçam de forma mais eficiente.
4. **Melhor uso de paralelismo**: O SQL Server pode distribuir operações entre diferentes partições e utilizar paralelismo de forma mais eficiente.

### Como funciona o particionamento no Azure SQL?

No Azure SQL, o particionamento segue a mesma abordagem do SQL Server local, usando **esquemas de partição** e **funções de partição**.

#### 1\. **Função de partição**

A função de partição define como os dados serão distribuídos entre as diferentes partições. Ela especifica os intervalos de valores que cada partição conterá.

Exemplo de criação de uma função de partição com base em uma coluna de data:  
  

CREATE PARTITION FUNCTION MyPartitionFunction (datetime) AS RANGE RIGHT FOR VALUES ('2023-01-01', '2023-06-01', '2023-12-01');  
  
Neste exemplo, a função divide a tabela em quatro partições, baseando-se nos intervalos de datas.

#### 2\. **Esquema de partição**

O esquema de partição define em quais arquivos (ou grupos de arquivos) as partições serão armazenadas. No Azure SQL Database, o armazenamento é gerenciado automaticamente pelo serviço, então, normalmente, você atribui todas as partições ao mesmo grupo de arquivos.

Exemplo de criação de um esquema de partição:  
  
CREATE PARTITION SCHEME MyPartitionScheme AS PARTITION MyPartitionFunction ALL TO (\[PRIMARY\]);  
  
  

Este esquema associa todas as partições ao grupo de arquivos principal (`PRIMARY`).

#### 3\. **Criar a tabela particionada**

Ao criar uma tabela particionada, você utiliza a função e o esquema de partição para definir como os dados serão distribuídos nas partições.

Exemplo de criação de uma tabela particionada:

#### CREATE TABLE Sales ( SaleID INT PRIMARY KEY, SaleDate DATETIME, Amount DECIMAL(10, 2) ) ON MyPartitionScheme(SaleDate);  
  
Neste exemplo, a tabela `Sales` é particionada com base na coluna `SaleDate`, utilizando o esquema e a função de partição previamente definidos.

### Como o particionamento afeta consultas no Azure SQL?

### 

Quando você consulta uma tabela particionada, o SQL Server pode otimizar o plano de execução, consultando apenas as partições relevantes, em vez de varrer toda a tabela. Isso é chamado de **eliminação de partições**.

Por exemplo, se sua tabela for particionada por data e você fizer uma consulta para buscar dados de vendas de junho de 2023:

### SELECT \* FROM Sales WHERE SaleDate \= '2023-06-15';  
  
O SQL Server acessará apenas a partição correspondente a junho de 2023, ignorando todas as outras partições.

### Desafios e boas práticas

# 

- **Chave de partição adequada**: Escolher uma boa coluna para ser a chave de partição é essencial. Colunas com alta cardinalidade (muitos valores distintos, como datas ou IDs sequenciais) são boas candidatas.
- **Reequilíbrio de partições**: Se os dados não forem distribuídos de forma equilibrada, uma partição pode acabar se tornando muito grande. Isso pode prejudicar o desempenho, então o particionamento deve ser bem planejado.
- **Manutenção**: Operações como reconstrução de índices podem ser aplicadas individualmente a partições, mas precisam ser planejadas para evitar sobrecarga em partições maiores.

### Considerações no Azure SQL Database

# 

1. **Azure SQL Database** tem suporte para particionamento, mas não para particionamento em múltiplos discos ou grupos de arquivos, já que o armazenamento é abstraído no serviço.
2. Em **Azure SQL Managed Instance**, você tem mais flexibilidade, como suporte a grupos de arquivos e opções mais avançadas de particionamento.

# Demo 1 criar uma tabela particionada por ano

1) Criar  o banco de dados

In [1]:
CREATE DATABASE [2.DemoTablesPartition]

Commands completed successfully.

Total execution time: 00:00:00.242

In [2]:
Use [2.DemoTablesPartition]

Commands completed successfully.

Total execution time: 00:00:00.001

### 1\. Criar a Função de Partição

A função de partição define como os dados serão distribuídos entre as partições. Neste caso, vamos criar uma função de partição que divide os dados por ano.

In [3]:
CREATE PARTITION FUNCTION YearPartitionFunction (DATE)
AS RANGE RIGHT FOR VALUES 
('2020-01-01', '2021-01-01', '2022-01-01', '2023-01-01');


Commands completed successfully.

Total execution time: 00:00:00.002

#### 2\. Criar o Esquema de Partição

Depois, você deve criar o esquema de partição:

In [4]:
CREATE PARTITION SCHEME YearPartitionScheme
AS PARTITION YearPartitionFunction
ALL TO ([PRIMARY]);


Partition scheme 'YearPartitionScheme' has been created successfully. 'PRIMARY' is marked as the next used filegroup in partition scheme 'YearPartitionScheme'.

Total execution time: 00:00:00.002

#### 3\. Criar a Tabela Particionada

Ao criar a tabela, você usará a função e o esquema de partição, garantindo que os dados sejam automaticamente alocados nas partições corretas.

In [8]:
drop TABLE Pedidos

Commands completed successfully.

Total execution time: 00:00:00.007

In [9]:
CREATE TABLE Pedidos
(
    ID_Pedido VARCHAR(20),
    Data_Pedido DATE,
    ID_Cliente VARCHAR(20),
    Segmento VARCHAR(50),
    Regiao VARCHAR(50),
    Pais VARCHAR(50),
    Product_ID VARCHAR(20),
    Categoria VARCHAR(50),
    SubCategoria VARCHAR(50),
    Total_Vendas DECIMAL(18, 2),
    Quantidade INT,
    Desconto DECIMAL(18, 2),
    Lucro DECIMAL(18, 2),
    Prioridade VARCHAR(20),
    PRIMARY KEY (ID_Pedido, Data_Pedido,Product_ID)  -- Incluindo Data_Pedido na chave primária
)
ON YearPartitionScheme(Data_Pedido);


Commands completed successfully.

Total execution time: 00:00:00.007

# Vamos importar o arquivo que está na pasta 
## dataset790MIL.csv

![Imagem de Exemplo](imagens/Import.png)


In [7]:
/****** Script for SelectTopNRows command from SSMS  ******/
SELECT TOP (1000) [ID_Pedido]
      ,[Data_Pedido]
      ,[ID_Cliente]
      ,[Segmento]
      ,[Regiao]
      ,[Pais]
      ,[Product_ID]
      ,[Categoria]
      ,[SubCategoria]
      ,[Total_Vendas]
      ,[Quantidade]
      ,[Desconto]
      ,[Lucro]
      ,[Prioridade]
  FROM [2.DemoTablesPartition].[dbo].[dataset790MIL]

(1000 rows affected)

Total execution time: 00:00:00.025

ID_Pedido,Data_Pedido,ID_Cliente,Segmento,Regiao,Pais,Product_ID,Categoria,SubCategoria,Total_Vendas,Quantidade,Desconto,Lucro,Prioridade
IN-2013-17853,2013-04-30,BS-11380,Corporativo,Qinghai,China,OFF-SU-10004462,Suprimentos,Supplies,4062,2,0,162,Medio
CA-2012-128139,2012-07-03,BD-11725,Consumidor,Kentucky,United States,FUR-CH-10003956,Moveis,Chairs,7098,1,0,49686,Medio
CA-2011-125612,2011-08-03,BK-11260,Consumidor,New York,United States,OFF-PA-10001019,Suprimentos,Paper,3996,2,0,187812,Medio
CA-2012-112452,2012-04-04,NC-18340,Consumidor,Michigan,United States,OFF-BI-10003350,Suprimentos,Binders,1276,2,0,58696,Critico
CA-2012-155600,2012-12-04,RO-19780,Consumidor,Tennessee,United States,OFF-AR-10003752,Suprimentos,Art,3696,4,2,12012,Medio
CA-2011-120775,2011-10-03,RD-19930,Consumidor,Texas,United States,FUR-FU-10000758,Moveis,Furnishings,31776,3,6,-190656,Alto
CA-2011-166730,2011-12-30,DD-13570,Consumidor,Tennessee,United States,OFF-ST-10002554,Suprimentos,Storage,39128,1,2,-88038,Medio
KG-2011-1040,2011-12-19,RM-9675,Home Office,Osh,Kyrgyzstan,OFF-CAM-10004269,Suprimentos,Envelopes,1695,1,0,609,Alto
CG-2013-7750,2013-09-16,OT-8730,Consumidor,Bandundu,Democratic Republic of the Congo,OFF-TEN-10000360,Suprimentos,Storage,2229,1,0,1068,Alto
MO-2013-7390,2013-01-29,MF-8250,Corporativo,Grand Casablanca,Morocco,OFF-STA-10001747,Suprimentos,Art,2505,1,0,15,Alto


## Removendo registros duplicados

In [10]:

WITH Dados AS (
SELECT  rn = ROW_NUMBER()
OVER( PARTITION BY ID_Pedido,Data_Pedido,ID_Cliente,
Product_ID ORDER BY ID_Pedido,Data_Pedido,ID_Cliente,
Product_ID),
[ID_Pedido]
      ,[Data_Pedido]
      ,[ID_Cliente]
      ,[Segmento]
      ,[Regiao]
      ,[Pais]
      ,[Product_ID]
      ,[Categoria]
      ,[SubCategoria]
      ,[Total_Vendas]
      ,[Quantidade]
      ,[Desconto]
      ,[Lucro]
      ,[Prioridade]
  FROM [2.DemoTablesPartition].[dbo].[dataset790MIL]

)
DELETE
 FROM Dados 
WHERE Dados.rn > 1

(0 rows affected)

Total execution time: 00:00:00.040

In [12]:

INSERT INTO dbo.Pedidos
(
    ID_Pedido,
    Data_Pedido,
    ID_Cliente,
    Segmento,
    Regiao,
    Pais,
    Product_ID,
    Categoria,
    SubCategoria,
    Total_Vendas,
    Quantidade,
    Desconto,
    Lucro,
    Prioridade
)
SELECT  [ID_Pedido]
      ,[Data_Pedido]
      ,[ID_Cliente]
      ,[Segmento]
      ,[Regiao]
      ,[Pais]
      ,[Product_ID]
      ,[Categoria]
      ,[SubCategoria]
      ,[Total_Vendas]
      ,[Quantidade]
      ,[Desconto]
      ,[Lucro]
      ,[Prioridade]
  FROM [2.DemoTablesPartition].[dbo].[dataset790MIL]

(51255 rows affected)

Total execution time: 00:00:00.217

In [17]:
SELECT * from Pedidos
where Data_Pedido = '2011-10-03'


(37 rows affected)

Total execution time: 00:00:00.047

ID_Pedido,Data_Pedido,ID_Cliente,Segmento,Regiao,Pais,Product_ID,Categoria,SubCategoria,Total_Vendas,Quantidade,Desconto,Lucro,Prioridade
AE-2011-9160,2011-10-03,PO-8865,Consumidor,'Ajman,United Arab Emirates,OFF-FEL-10001405,Suprimentos,Storage,82674.00,2,7.00,-157086.00,Medio
AE-2011-9160,2011-10-03,PO-8865,Consumidor,'Ajman,United Arab Emirates,TEC-EPS-10004171,Tecnologia,Machines,78408.00,6,7.00,-88992.00,Medio
CA-2011-108903,2011-10-03,DO-13435,Consumidor,Ohio,United States,OFF-AR-10004010,Suprimentos,Art,55984.00,2,2.00,41988.00,Alto
CA-2011-108903,2011-10-03,DO-13435,Consumidor,Ohio,United States,OFF-EN-10004030,Suprimentos,Envelopes,1448.00,5,2.00,4887.00,Alto
CA-2011-108903,2011-10-03,DO-13435,Consumidor,Ohio,United States,TEC-AC-10003023,Tecnologia,Accessories,142488.00,3,2.00,-35622.00,Alto
CA-2011-120775,2011-10-03,RD-19930,Consumidor,Texas,United States,FUR-FU-10000758,Moveis,Furnishings,31776.00,3,6.00,-190656.00,Alto
CA-2011-120775,2011-10-03,RD-19930,Consumidor,Texas,United States,OFF-BI-10002609,Suprimentos,Binders,1788.00,3,8.00,-30396.00,Alto
CA-2011-120775,2011-10-03,RD-19930,Consumidor,Texas,United States,OFF-FA-10000254,Suprimentos,Fasteners,15072.00,4,2.00,-3768.00,Alto
CA-2011-120775,2011-10-03,RD-19930,Consumidor,Texas,United States,OFF-FA-10002676,Suprimentos,Fasteners,4344.00,3,2.00,8688.00,Alto
CA-2011-120775,2011-10-03,RD-19930,Consumidor,Texas,United States,OFF-LA-10002271,Suprimentos,Labels,4928.00,2,2.00,17248.00,Alto


### Vantagens dessa Abordagem

1. **Futuras Expansões**: Se você quiser adicionar mais anos, você pode facilmente modificar a função de partição sem a necessidade de alterar as inserções de dados.
    
2. **Flexibilidade**: Com essa configuração, você pode também considerar dados que não estão limitados a apenas alguns anos, tornando a estrutura mais flexível e adaptável a mudanças no seu modelo de dados.
    
3. **Eliminação de Partições**: Consultas baseadas em intervalos de data se beneficiam da eliminação de partições, melhorando o desempenho.

### Atualização da Função de Partição

Para adicionar novos anos à função de partição, você precisará usar `ALTER PARTITION FUNCTION` para adicionar novos limites:

In [32]:
SELECT 
    f.name,f.type_desc,p.*
FROM 
    sys.partition_range_values p 
    join sys.partition_functions f on f.function_id = p.function_id



(4 rows affected)

Total execution time: 00:00:00.011

name,type_desc,function_id,boundary_id,parameter_id,value
YearPartitionFunction,RANGE,65536,1,1,2020-01-01 00:00:00
YearPartitionFunction,RANGE,65536,2,1,2021-01-01 00:00:00
YearPartitionFunction,RANGE,65536,3,1,2022-01-01 00:00:00
YearPartitionFunction,RANGE,65536,4,1,2023-01-01 00:00:00


In [34]:
-- Se a partição para 2011 não existir, adicione-a
ALTER PARTITION FUNCTION YearPartitionFunction()
ADD RANGE RIGHT FOR VALUES ('2020-01-01 00:00:00');

-- Agora você pode mesclar
ALTER PARTITION FUNCTION YearPartitionFunction()
MERGE RANGE ('2020-01-01 00:00:00');

: Msg 156, Level 15, State 1, Line 2
Incorrect syntax near the keyword 'ADD'.

: Msg 102, Level 15, State 1, Line 2
Incorrect syntax near 'RANGE'.

Total execution time: 00:00:00

Ou, se precisar apenas adicionar um novo intervalo:

In [20]:
ALTER PARTITION FUNCTION YearPartitionFunction()
ADD RANGE ('2024-01-01');

: Msg 156, Level 15, State 1, Line 2
Incorrect syntax near the keyword 'ADD'.

: Msg 102, Level 15, State 1, Line 2
Incorrect syntax near 'RANGE'.

Total execution time: 00:00:00

(4 rows affected)

Total execution time: 00:00:00.012

function_id,boundary_id,parameter_id,value
65536,1,1,2020-01-01 00:00:00
65536,2,1,2021-01-01 00:00:00
65536,3,1,2022-01-01 00:00:00
65536,4,1,2023-01-01 00:00:00
